In [ ]:
import warnings
warnings.filterwarnings("ignore")


In [ ]:
from keras.datasets import cifar10
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten, Activation,  MaxPooling2D, BatchNormalization
from keras import regularizers
from keras.utils import np_utils
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
# loading the dataset
(X_train, y_train), (X_test, y_test) = cifar10.load_data()

# # building the input vector from the 32x32 pixels
X_train = X_train.reshape(X_train.shape[0], 32, 32, 3)
X_test = X_test.reshape(X_test.shape[0], 32, 32, 3)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

# normalizing the data to help with the training
X_train /= 255
X_test /= 255

# expand the input size by 3*3, so the input now is 96*96
X_train = np.repeat(X_train,3,axis=1)
X_train = np.repeat(X_train,3,axis=2)
X_test = np.repeat(X_test,3,axis=1)
X_test = np.repeat(X_test,3,axis=2)



# one-hot encoding using keras' numpy-related utilities
n_classes = 10
print("Shape before one-hot encoding: ", y_train.shape)
Y_train = np_utils.to_categorical(y_train, n_classes)
Y_test = np_utils.to_categorical(y_test, n_classes)
print("Shape after one-hot encoding: ", Y_train.shape)

# building a linear stack of layers with the sequential model
model = Sequential()
L2_coeff = 0.0005
# convolutional layer
model.add(Conv2D(75, (3, 3), padding='same', activation='relu', input_shape=[96,96,3],kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
# convolutional layer
model.add(Conv2D(50, kernel_size=(3,3), padding='same', activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

model.add(Conv2D(50, kernel_size=(3,3), padding='same', activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(MaxPool2D(pool_size=(2,2)))

# flatten output of conv
model.add(Flatten())

# hidden layer
model.add(Dense(250, activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
model.add(Dense(250, activation='relu',kernel_regularizer=regularizers.l2(L2_coeff)))
model.add(BatchNormalization())
# output layer
model.add(Dense(10, activation='softmax'))

# compiling the sequential model
model.compile(loss='categorical_crossentropy', metrics=['accuracy'], optimizer='adam')

# model.fit(X_train, Y_train, batch_size=250, epochs=50, validation_data=(X_test, Y_test))
datagen = ImageDataGenerator(
            rotation_range=25,  # randomly rotate images in the range (degrees, 0 to 180)
            width_shift_range=0.15,  # randomly shift images horizontally (fraction of total width)
            height_shift_range=0.15,  # randomly shift images vertically (fraction of total height)
            horizontal_flip=True,  # randomly flip images
            vertical_flip=True)  # randomly flip images
datagen.fit(X_train)
model.fit(datagen.flow(X_train, Y_train, batch_size=100), steps_per_epoch=500, epochs=50, validation_data=(X_test, Y_test))

Shape before one-hot encoding:  (50000, 1)
Shape after one-hot encoding:  (50000, 10)
Epoch 1/50
500/500 [==============================] - 68s 128ms/step - loss: 2.0979 - accuracy: 0.4198 - val_loss: 2.5100 - val_accuracy: 0.3051
Epoch 2/50
500/500 [==============================] - 63s 127ms/step - loss: 1.7855 - accuracy: 0.5035 - val_loss: 1.8047 - val_accuracy: 0.4962
Epoch 3/50
500/500 [==============================] - 64s 127ms/step - loss: 1.7103 - accuracy: 0.5363 - val_loss: 2.1660 - val_accuracy: 0.4205
Epoch 4/50
500/500 [==============================] - 63s 127ms/step - loss: 1.6917 - accuracy: 0.5582 - val_loss: 2.3136 - val_accuracy: 0.4510
Epoch 5/50
500/500 [==============================] - 63s 127ms/step - loss: 1.6799 - accuracy: 0.5745 - val_loss: 1.6335 - val_accuracy: 0.5938
Epoch 6/50
500/500 [==============================] - 63s 127ms/step - loss: 1.6644 - accuracy: 0.5856 - val_loss: 1.8390 - val_accuracy: 0.5468
Epoch 7/50
500/500 [========================

In [ ]:

model.fit(X_train, Y_train, batch_size=100, epochs=1, validation_data=(X_test, Y_test))

500/500 [==============================] - 32s 64ms/step - loss: 1.0286 - accuracy: 0.7634 - val_loss: 1.1439 - val_accuracy: 0.7179


In [ ]:
model.save('simple_cnn_expanded3.h5')


In [ ]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 96, 96, 75)        2100      
_________________________________________________________________
batch_normalization (BatchNo (None, 96, 96, 75)        300       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 96, 96, 50)        33800     
_________________________________________________________________
batch_normalization_1 (Batch (None, 96, 96, 50)        200       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 48, 48, 50)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 48, 48, 50)        22550     
_________________________________________________________________
batch_normalization_2 (Batch (None, 48, 48, 50)        2